In [1]:
from MelodyLSTMEmbV2 import MelodyLSTMEmb
import torch
import pickle
import numpy as np
import pretty_midi

In [2]:
num_epochs = 100
learning_rate = 1e-6

input_size = 56 #number of features
hidden_size = 512 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

output_size = 36 #number of output classes 

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)
mlstm = MelodyLSTMEmb(input_size, hidden_size, output_size, num_layers, device)
mlstm.load_state_dict(torch.load('../models/muse2_t10000_bce_w20/checkpoint-20.pth')) 
mlstm.eval()
mlstm.to(device)

Device cuda


MelodyLSTMEmb(
  (embedding): Embedding(51, 10)
  (lstm): LSTM(56, 512, batch_first=True)
  (fc_1): Linear(in_features=512, out_features=256, bias=True)
  (fc): Linear(in_features=256, out_features=36, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [4]:
triad_dict = {
    'C': (0, 4, 7),
    'C#': (1, 5, 8),
    'D': (2, 6, 9),
    'D#': (3, 7, 10), 
    'E': (4, 8, 11),
    'F': (0, 5, 9),
    'F#': (1, 6, 10),
    'G': (2, 7, 11),
    'G#': (0, 3, 8),
    'A': (1, 4, 9),
    'A#': (2, 5, 10),
    'B': (3, 6, 11), 
    'Cm': (0, 3, 7),
    'C#m': (1, 4, 8),
    'Dm': (2, 5, 9),
    'D#m': (3, 6, 10),
    'Em': (4, 7, 11),
    'Fm': (0, 5, 8),
    'F#m': (1, 6, 9),
    'Gm': (2, 7, 10),
    'G#m': (3, 8, 11),
    'Am': (0, 4, 9),
    'A#m': (1, 5, 10),
    'Bm': (2, 6, 11)
}

In [5]:
with open('../data/chords_reduced/CHORD_TO_EMB.pickle', 'rb') as f:
    CHORD_TO_EMB = pickle.load(f)

In [6]:
with open('../data/chords/CHORD_DICT.pickle', 'rb') as f: 
    CHORD_DICT = pickle.load(f)

In [7]:
EMB_TO_CHORD = {v: k for k, v in CHORD_TO_EMB.items()}

In [8]:
def get_embedding_chord(chord):
    triad = triad_dict[chord]
    triad = triad[0] * 10000 + triad[1] * 100 + triad[2]
    idx = CHORD_DICT.index(triad)
    emb = CHORD_TO_EMB[idx]
    return emb

In [15]:
def select_notes(p):
    num = np.random.randint(1, 4)
    possible = p.detach().cpu().numpy().reshape(-1)
    possible = np.asarray(possible > 0.2).nonzero()[0]
    print(possible)
    return np.random.choice(len(possible), replace=False, 

In [10]:
def generate_melody_compass(chord, chord_next, prev=None, ts=8):
    notes = []
    if prev == None:
        prev = torch.zeros(38)
        prev[-1] = chord_next
        prev[-2] = chord
        prev = torch.reshape(prev, (1, 1, -1))
    
    for _ in range(ts):
        prev = prev.to(device)
        p = mlstm(prev)
        notes.append(select_notes(p))
        
        last = torch.zeros_like(prev[:,-1,:])
        last = torch.reshape(last, (1, 1, -1))
        last[:,:,notes[-1]] = 1
        last[:, :, -2] = chord
        last[:, :, -1] = chord_next
        print(last)
        prev = torch.cat((prev, last), dim=1)
        if prev.size(1) == 4:
            prev = prev[:, 1:, :]
            
    return notes, prev

In [11]:
def generate_melody_for_chords(chords):
    notes = []
    prev = None
    chords += [chords[0]]
    for i in range(len(chords)-1):
        notes_i, prev = generate_melody_compass(chords[i], chords[i + 1], prev)
        notes += notes_i
    return notes

In [18]:
chord_progression = ['C', 'G', 'Am', 'F']
chord_progression_ids = [get_embedding_chord(chord) for chord in chord_progression]

In [19]:
chord_progression_ids

[0, 1, 3, 2]

In [20]:
notes = generate_melody_for_chords(chord_progression_ids)

[19 21 24]
tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 1.]]], device='cuda:0')
[19 21 24 28]
tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          0., 0., 0., 1.]]], device='cuda:0')
[19 21 24 28]
tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          0., 0., 0., 1.]]], device='cuda:0')
[19 21 24 28]
tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          0., 0., 0., 1.]]], device='cuda:0')
[19 21 24 28]
tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 1., 0., 1., 0., 0., 1., 0., 0

In [89]:
def write_melody_to_piano_roll(notes, piano_roll):
    for i, note in enumerate(notes):
        piano_roll[note + 36, i] = 100
    
    return piano_roll

In [90]:
piano_roll_notes = np.zeros((128, 32))

In [91]:
def write_piano_roll_to_midi(piano_roll, midi, pid):
    instrument = pretty_midi.Instrument(program=pid)
    
    # add notes to the instrument object
    for note_idx, time_slice in enumerate(piano_roll.T):
        note_numbers = np.nonzero(time_slice)[0]
        for note_number in note_numbers:
            note_start = note_idx / 4.0
            note_end = (note_idx + 1) / 4.0
            note_velocity = int(time_slice[note_number])
            note = pretty_midi.Note(
                velocity=note_velocity,
                pitch=note_number,
                start=note_start,
                end=note_end
            )
            instrument.notes.append(note)

    # add the instrument object to the MIDI object
    midi.instruments.append(instrument)
    
    return midi

In [92]:
paino_roll_notes = write_melody_to_piano_roll(notes, piano_roll_notes)

In [93]:
def save_piano_roll_to_midi(piano_roll_notes, filename):
    # create a PrettyMIDI object
    midi = pretty_midi.PrettyMIDI()
    #midi = write_piano_roll_to_midi(piano_roll_chords, midi, 0)
    midi = write_piano_roll_to_midi(piano_roll_notes, midi, 0)
    
    # write the MIDI object to a file
    midi.write(filename)

In [94]:
save_piano_roll_to_midi(piano_roll_notes, 'muse2_poly_t2000_75.mid')